## XFOIL analysis with python UI

    Commend "LOAD"
            "GDES" switches the XFOIL to geometry design stage
            "CADD" adds corner points to satisfy user specified angle threshold.
            "PANEL" regenerates vertices
            "OPER" switches the program to aerodynamic analysis mode
            "ALFA 5" angle of attack = 5
        

In [1]:
import subprocess as sp
import shutil
import sys
import string
import shutil


In [2]:
import os
from pathlib import Path
import sched, time
def boostup(loc):
    p = Path(loc)
    #if the directory is there, delete it
    if p.is_dir():
        shutil.rmtree(loc)
    #when the directory is still ther, sleep
    while p.is_dir():
        time.sleep(2)
    #create da directory
    os.makedirs(loc)
    pass

## xfoil commend
    LOAD dat/e1211-il.dat	Load the dat file
    MDES	Go to the MDES menu
    FILT	Smooth any variations in the dat file data
    EXEC	Execute the smoothing
            Back to main menu
    PANE	Set the number and location of the airfoil points for analysis
    OPER	Go to the OPER menu
    ITER 70	Max number of iterations set to 70 for convergence
    RE 50000	Set Reynolds number (required?)
    VISC 50000	Set viscous calculation with Reynolds number
    PACC	Start polar output file
    polar/e1211-il_50000.txt	The output polar file name
            No dump file
    ALFA 0	Calculate lift and drag at 0° angle of attack
    ALFA 0.25	... 0.25°
    ALFA 0.5	... 0.5° ...
    ...	...more alpha calculations here ...
    ALFA 3.5	At 3.5° no convergence
    ALFA 3.5	... try again ...
    ALFA 3.5	... and again
    INIT	Run INIT to reinitialise
    ALFA 3.75	Skip to 3.75°
    ...	...rest of alpha calculations here ...
    PACC	Close polar file
    VISC	Reinitialise viscous calculation (required?)
    Down to main menu
    QUIT	Exit Xfoil

In [3]:
def script(airfoil_loc, airfoil, data_loc, AOT, RE):
    load_script = ['load '+airfoil_loc+airfoil+".txt", 
            airfoil,
            'MDES',
            'FILT',
            'EXEC',
            ' ',
            'PANE',
            'OPER',
            'ITER 150']       
    #define the output file name (different reynolds number has different file)
    data_file_name = data_loc+'/'+str(RE)+'.txt'
    #check and create file
    constant_set = ['RE ' + str(RE),
                    'VISC ' +str(RE),
                    'PACC', 
                    data_file_name,
                    ' ']
    #analyse different angle of attack
    alpha = ['ALFA '+str(a) for a in AOT]
    init = constant_set + alpha + ['PACC', ' ',]
    load_script += init
    load_script += [' ', ' ', 'QUIT\n']
    return load_script

In [4]:
def file_mining(file):
    #open file and convert them into list
    f = list(open(file, 'r'))
    #only select those data (need to be test if there is a buy)
    f = f[12:]
    #contains a list contains all the [alpha, CL]
    output = [i.split()[:2] for i in f]
    #since now output value elements are string, we need to con conver them to float
    output = [list(map(float, row)) for row in output]
    return output

def foil_mining(data,gen,airfoil,reys):
    #make the data directory of the airfoil
    loc = data+gen+'/'+airfoil+'/'
    outputs = []
    for rey in reys:
        #get the output for each reynolds number
        output = file_mining(loc+rey)
        #put rey data with angle value
        outputs.append([rey,output])
    #example: [rey, [*angle, *cl, *cd]]
    return outputs

    class mining():
        '''
            airfoil: the series number of airfoil
            rey: reynolds number
            data_loc: the location of data directory
            gen: which generation'''
        def __init__(self, airfoil, rey, data_loc, gen):
            self.airfoil = airfoil
            self.rey = rey
            self.data_loc = data_loc
            self.gen = gen

        def file_mining(file):
            #open file and convert them into list
            f = list(open(file, 'r'))
            #only select those data (need to be test if there is a buy)
            f = f[12:]
            #contains a list contains all the [alpha, CL, CD]
            output = [i.split()[:3] for i in f]
            #since now output value elements are string, we need to con conver them to float
            output = [list(map(float, row)) for row in output]
            output = {row[0]: row[1:] for row in output}
            #the output is a dictionary: {angle: [CL, CD]}
            return output

        def dir_mining(self):
            loc = self.data_loc + self.gen
            for model in airfoil:


## Fitness function idea
    1: reynolds number score:   rey: weights
                                0.5: 1
                                0.2: 2
    
    2: angle data score:        angle: weights
                                13: 2
                                
                                
    3: sum up those points
    
    
    20% thickness
    more importantly improve lift

In [5]:
import numpy as np
def heuristic(data):
    #contains: airfoil info, rey, angles, data
    #ratio = {angle: cl/cd}
    for i in data:
        angleP = heuristic_angle(i[1])
        reyP = heuristic_rey(i[0])
        
def sigmoid(x):
    return 1/(1 + np.exp(-x))

def data_check(cl):
    if max(cl) > 1.4:
        return False
    else:
        return True
        
def heuristic_rey(rey):
    '''angle 8,9,10 value weights 10'''
    pass

def heuristic_angle(datas):
    data = np.transpose(datas)
    if data_check(data[1]):
        weights = 5 * sigmoid(data[0] / 2.5) + 0.01
        point = data[1] * weights
    else:
        point = 6
    return point        

In [6]:
a = [1,2,3]
a = np.array(a)
a * a

array([1, 4, 9])

In [7]:
#constants setting
airfoil_loc = 'airfoil/'
data_loc = 'data/'
angle_range = [-10, -8, 0,2,4,6,8,10]
rey_range = [100000, 200000, 300000,400000,500000]

In [16]:
#reboost the system everytime it starts
boostup(data_loc)

In [17]:
def execute(airfoil_loc, airfoil, data_loc, angles, rey_range):
    #create a directory for each airfoil
    filedir = data_loc+airfoil
    os.makedirs(filedir)
    for RE in rey_range:
        #coupling xfoil into python
        ps = sp.Popen(['xfoil.exe'],
                      stdin=sp.PIPE,
                      stdout=None,
                      stderr=None,
                      encoding='utf8')
        #execute the script
        res = ps.communicate('\n'.join(script(airfoil_loc,airfoil, filedir, angle_range, RE)))
   

In [18]:
execute('airfoil/1gen/', 'S813', 'data/1gen/', angle_range, rey_range)